In [21]:
import torch
from torch._export import capture_pre_autograd_graph

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

weight_path = '/home/aa35037123/Wesley/edge_ai/lab3/mobilenetv2_0.963.pth'
model = torch.load(weight_path, map_location=torch.device('cpu'))
model.to('cpu')
model.eval()
example_inputs = (torch.randn(1, 3, 224, 224),)
model = capture_pre_autograd_graph(model, example_inputs)
# Step 2. quantization
from torch.ao.quantization.quantize_pt2e import (
  prepare_pt2e,
  convert_pt2e,
)

from torch.ao.quantization.quantizer import (
  XNNPACKQuantizer,
  get_symmetric_quantization_config,
)
# backend developer will write their own Quantizer and expose methods to allow
# users to express how they
# want the model to be quantized
quantizer = XNNPACKQuantizer().set_global(get_symmetric_quantization_config())
model = prepare_pt2e(model, quantizer)

# calibration omitted

model = convert_pt2e(model)
# we have a model with aten ops doing integer computations when possible

In [22]:
import torch
from torchvision.models import mobilenet_v2
# from torchvision.models.quantization import mobilenet_v2
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import os
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import copy

weight_path = '/home/aa35037123/Wesley/edge_ai/lab3/mobilenetv2_0.963.pth'
model = torch.load(weight_path, map_location=torch.device('cpu'))
model.to('cpu')
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [23]:
import os
import sys
import time
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.models.resnet import resnet18
import torchvision.transforms as transforms

# Set up warnings
import warnings
warnings.filterwarnings(
    action='ignore',
    category=DeprecationWarning,
    module=r'.*'
)
warnings.filterwarnings(
    action='default',
    module=r'torch.ao.quantization'
)

# Specify random seed for repeatable results
_ = torch.manual_seed(191009)

weight_path = '/home/aa35037123/Wesley/edge_ai/lab3/mobilenetv2_0.963.pth'
model_to_quantize = torch.load(weight_path, map_location=torch.device('cpu'))
model_to_quantize.to('cpu')
model_to_quantize.eval()

In [24]:
from torch._export import capture_pre_autograd_graph
exported_model = capture_pre_autograd_graph(model_to_quantize, example_inputs)

In [25]:
from torch.ao.quantization.quantizer.xnnpack_quantizer import (
  XNNPACKQuantizer,
  get_symmetric_quantization_config,
)
quantizer = XNNPACKQuantizer()
quantizer.set_global(get_symmetric_quantization_config())

Files already downloaded and verified
Files already downloaded and verified
device: cpu
Size (MB): 9.169412


In [26]:
quantizer.set_global(qconfig_opt).set_object_type(torch.nn.Conv2d, qconfig_opt).set_object_type(torch.nn.functional.linear, qconfig_opt).set_module_name("foo.bar", qconfig_opt)

/home/aa35037123/miniconda3/envs/lab3/lib/python3.10/site-packages/torch/ao/quantization/observer.py:1263: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(
/home/aa35037123/miniconda3/envs/lab3/lib/python3.10/site-packages/torch/ao/quantization/utils.py:339: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


GraphModule()

In [27]:
prepared_model = prepare_pt2e(exported_model, quantizer)
print(prepared_model.graph)

In [ ]:
def evaluate_model(model, data_loader,device):

    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy}%')
    return accuracy


In [ ]:
def prepare_data(batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to match MobileNet input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    train_set = CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = CIFAR10(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False,drop_last=True)
    return train_loader, test_loader

batch_size = 16
train_loader, test_loader = prepare_data(batch_size)

In [28]:
def calibrate(model, data_loader):
    model.eval()
    with torch.no_grad():
        for images, labels in data_loader:
            model(images)
calibrate(prepared_model, test_loader)  # run calibration on sample data

Accuracy of the model on the test images: 89.83%


In [ ]:
ptq_quantized_model = convert_pt2e(prepared_model)
print(ptq_quantized_model)

In [30]:
# Export the model and Save ExportedProgram
pt2e_quantized_model_file_path =  "./mobilenet_quantized.pt"
# capture the model to get an ExportedProgram
example_inputs = (torch.randn(1, 3, 224, 224),)
quantized_ep = torch.export.export(ptq_quantized_model, example_inputs)
# use torch.export.save to save an ExportedProgram
torch.export.save(quantized_ep, pt2e_quantized_model_file_path)

In [31]:
loaded_quantized_ep = torch.export.load(pt2e_quantized_model_file_path)
loaded_quantized_model = loaded_quantized_ep.module()
acc = evaluate_model(loaded_quantized_model, test_loader, device)

ImportError: cannot import name 'get_lifted_tensor_constant' from 'torch._export.utils' (/home/aa35037123/miniconda3/envs/lab3/lib/python3.10/site-packages/torch/_export/utils.py)